In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap

# -----------------
# 1. Load Model and Data
# -----------------
# Updated file paths: "scaled" instead of "pca"
model_path = "folds_transformed_models/fold_4/scaled/MIL4 (sliderNeutralPos)/rf/rf_MIL4 (sliderNeutralPos).pkl"
data_path = "folds_transformed/fold_4_train_scaled.csv"

# Load the pre-trained random forest classifier
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load the CSV data
df = pd.read_csv(data_path)

# -----------------
# 2. Infer Target and Select Features
# -----------------
# Infer the target column from the model file name.
model_file_name = os.path.basename(model_path)  # e.g., "rf_MIL3 (sliderNeutralPos).pkl"
inferred_target = model_file_name.replace("rf_", "").replace(".pkl", "")
print("Inferred target column:", inferred_target)

# Drop non-feature columns: remove any column that has "slider" in its name, unless it's the target.
feature_cols = [col for col in df.columns if (col == inferred_target or "slider" not in col.lower())]

# From the features, remove the target column so that only predictors remain.
X = df[feature_cols].drop(columns=[inferred_target])
y = df[inferred_target]

# Select a subset of samples for SHAP analysis to reduce computation time
X_sample = X.sample(n=100, random_state=42)  # Adjust sample size as needed


In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt

# -----------------
# 1. Load Model and Data
# -----------------
model_path = "folds_transformed_models/fold_4/scaled/MIL4 (sliderNeutralPos)/rf/rf_MIL4 (sliderNeutralPos).pkl"
data_path = "folds_transformed/fold_4_train_scaled.csv"

# Load the pre-trained classifier (Random Forest)
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load the CSV data
df = pd.read_csv(data_path)

# -----------------
# 2. Infer Target and Select Features
# -----------------
# Extract target variable from the model filename
model_file_name = os.path.basename(model_path)
inferred_target = model_file_name.replace("rf_", "").replace(".pkl", "")
print("Inferred target column:", inferred_target)

# Select feature columns, ignoring non-relevant columns
feature_cols = [col for col in df.columns if (col == inferred_target or "slider" not in col.lower())]
X = df[feature_cols].drop(columns=[inferred_target])
y = df[inferred_target]

# -----------------
# 3. SHAP Analysis for Multi-Class Classification
# -----------------

# Select a subset of 100 samples for analysis to reduce computation time
X_sample = X.sample(n=100, random_state=42)

# Ensure the model supports predict_proba (required for multi-class SHAP)
if hasattr(model, "predict_proba"):
    explainer = shap.Explainer(model.predict_proba, X_sample)
else:
    explainer = shap.Explainer(model, X_sample)

# Compute SHAP values
shap_values = explainer(X_sample)

# Print SHAP values shape: (n_samples, n_features, n_classes)
print("SHAP values shape:", shap_values.values.shape)

# -----------------
# 4. Automatically Infer Available Classes
# -----------------

# Get the number of classes from SHAP values
num_classes = shap_values.values.shape[2]  # Third axis represents classes

# Print detected class indices
available_classes = list(range(num_classes))  # Classes are indexed from 0 to (num_classes-1)
print(f"Detected available classes: {available_classes}")

# -----------------
# 5. Multi-Class SHAP Summary in One Plot
# -----------------

feature_names = list(X_sample.columns)  # Ensure feature names are correctly passed as a list

# Aggregate SHAP values across all classes (e.g., mean effect per feature)
shap_values_mean = shap_values.values.mean(axis=2)  # Shape: (n_samples, n_features)

# Plot the aggregated SHAP values
shap.summary_plot(
    shap_values_mean,
    X_sample,
    feature_names=feature_names
)


In [ ]:
# -----------------
# 4. LIME Analysis for Multi-Class Classification
# -----------------
import lime
import lime.lime_tabular

# Initialize the LIME explainer
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X.values,
    feature_names=X.columns.tolist(),
    class_names=[str(cls) for cls in model.classes_],  # Ensure class names are string labels
    mode='classification',
    verbose=True,
    random_state=42
)

# Choose an instance index to explain
i = 0  # Change this index to explain different samples
instance = X.iloc[i].values

# Get available classes (from model output)
num_classes = len(model.classes_)  # Automatically determine number of classes
print(f"Available classes: {model.classes_}")

# Plot LIME explanations for each class
for class_idx in range(num_classes):
    print(f"Explaining instance {i} for class {model.classes_[class_idx]}")

    # Explain instance for the specific class
    exp = lime_explainer.explain_instance(
        instance,
        model.predict_proba,
        num_features=10,  # Adjust number of features to display
        labels=(class_idx,)  # Specify the class index
    )

    # Plot LIME explanation for this class
    fig = exp.as_pyplot_figure(label=class_idx)
    plt.title(f"LIME Explanation for Instance {i} (Class {model.classes_[class_idx]})")
    plt.show()


In [ ]:
# -----------------
# 5. MAPIE for Classification Prediction Sets
# -----------------
from mapie.classification import MapieClassifier

alpha = 0.05  # significance level for prediction sets
mapie = MapieClassifier(estimator=model, cv="prefit", method="score")
mapie.fit(X, y)  # calibrate MAPIE (this does not re-fit your model)

# Obtain prediction sets for each instance in X
y_pred, prediction_sets = mapie.predict(X, alpha=alpha)

# Calculate the coverage: fraction of instances where the true label is in the prediction set.
coverage = np.mean([y.iloc[i] in prediction_sets[i] for i in range(len(y))])
print(f"Prediction set coverage: {coverage:.3f}")

# Inspect the prediction sets for a few examples:
for i in range(5):
    print(f"Instance {i}: True label = {y.iloc[i]}, Prediction set = {prediction_sets[i]}")

# Since prediction sets are lists of class labels, you can visualize the distribution of their sizes.
set_sizes = [len(pset) for pset in prediction_sets]

plt.figure(figsize=(8, 4))
plt.hist(set_sizes, bins=np.arange(1, max(set_sizes)+2)-0.5, edgecolor='black')
plt.xlabel("Prediction Set Size")
plt.ylabel("Frequency")
plt.title("Distribution of MAPIE Prediction Set Sizes")
plt.show()